### Preparando o código

In [35]:
#importando as bibliotecas
import pandas as pd
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np
import plotly.graph_objects as go

In [36]:
#importando a planilha

caminho = r'/content/vendas_distribuidoras_anp 1.xlsx'


gasolina = pd.read_excel(caminho, sheet_name='gasolina')
etanol = pd.read_excel(caminho, sheet_name='etanol')
diesel = pd.read_excel(caminho, sheet_name='diesel')

produtos = [gasolina, etanol, diesel]

total_consumo_gasolina = pd.DataFrame()
total_consumo_etanol = pd.DataFrame()
total_consumo_diesel = pd.DataFrame()





### Primeira questão

In [37]:
#Como o consumo de cada tipo de combustível está distribuído entre os estados brasileiros?

# Criando dataframes vazios com as variáveis de interesse para serem populados pelo loop


total_consumo_gasolina = pd.DataFrame()
total_consumo_etanol = pd.DataFrame()
total_consumo_diesel = pd.DataFrame()

#consumo do ultimo ano, que se encontra com dados incompletos e deve ser analisado separadamente

consumo_2021_gasolina = pd.DataFrame()
consumo_2021_etanol = pd.DataFrame()
consumo_2021_diesel = pd.DataFrame()

#Consumo médio
consumo_medio_gasolina = pd.DataFrame()
consumo_medio_etanol = pd.DataFrame()
consumo_medio_diesel = pd.DataFrame()



for produto in produtos:

  #Agrupando por ente federativo, somando e tirando a média
  total_consumo = produto.groupby(produto['regiao']).sum()
  media_consumo = produto.groupby(produto['regiao']).mean()

  # Filtrando e transpondo as colunas
  total_consumo = total_consumo.loc[:,'2016':].T
  total_consumo.index.rename(name='Ano', inplace=True)

  # Criando uma variável com os dados de 2021
  consumo_2021 = total_consumo.iloc[-1]

   # Retirando os dados de 2021
  total_consumo = total_consumo.iloc[:-1]

   # dividindo por 10000 e arrendondando algumas virgulas para melhorar o aspecto dos resultados
  total_consumo = round(total_consumo/10000,2)


  #tirando a média dos últimos 5 anos
  consumo_medio = total_consumo.mean()
  consumo_medio_df = pd.DataFrame(consumo_medio)
  consumo_medio_df.columns = ['Valores (%)']

  #Adicionando os dados ao dataframe para cada um dos produtos

  if produto is gasolina:
    total_consumo_gasolina = total_consumo
    consumo_2021_gasolina = consumo_2021
    consumo_medio_gasolina = consumo_medio_df

  elif produto is etanol:
    total_consumo_etanol = total_consumo
    consumo_2021_etanol = consumo_2021
    consumo_medio_etanol = consumo_medio_df

  elif produto is diesel:
    total_consumo_diesel = total_consumo
    consumo_2021_diesel = consumo_2021
    consumo_medio_diesel = consumo_medio_df






In [38]:
#Printando os dfs de consumo médio


print(f'Consumo médio de gasolina: {display(consumo_medio_gasolina)}' )


print(f'Consumo médio de etanol: {display(consumo_medio_etanol)}')

print(f'Consumo médio de diesel: {display(consumo_medio_diesel)}')



,Valores (%)
regiao,
br,3990.180
df,112.524
go,131.722
ma,94.056
mg,383.914
mt,54.788
pa,116.612
sp,880.258
to,36.206


Consumo médio de gasolina: None


,Valores (%)
regiao,
br,1788.286
df,11.590
go,138.154
ma,3.692
mg,226.776
mt,80.612
pa,4.406
sp,956.298
to,3.062


Consumo médio de etanol: None


,Valores (%)
regiao,
br,5589.018
df,36.800
go,273.506
ma,135.304
mg,687.980
mt,284.950
pa,233.458
sp,1212.992
to,97.668


Consumo médio de diesel: None


### Analise do consumo de *gasolina*

In [39]:

# consumo médio proporcional

consumo_proporcional_gasolina =(consumo_medio_gasolina/consumo_medio_gasolina.iloc[0])

onsumo_proporcional_gasolina = consumo_proporcional_gasolina.sort_values('Valores (%)', ascending=False)

consumo_proporcional_gasolina = consumo_proporcional_gasolina.drop('br').sort_values(ascending=False, by='Valores (%)')
consumo_proporcional_gasolina = consumo_proporcional_gasolina*100


consumo_proporcional_gasolina


px.bar(consumo_proporcional_gasolina, y='Valores (%)',title= 'consumo de Gasolina')







In [45]:
# Crescimento e Tamanho do mercado


crescimento_mercado_gasolina = total_consumo_gasolina/total_consumo_gasolina.iloc[0]
crescimento_mercado_gasolina




# Criando o gráfico
fig = go.Figure()

# Adicionando linhas para cada coluna de valores
for col in crescimento_mercado_gasolina.columns[1:]:
    fig.add_trace(go.Scatter(
        x = crescimento_mercado_gasolina.index,
        y = crescimento_mercado_gasolina[col],
        mode='lines+markers',
        name=col
    ))

# Configuração do layout
fig.update_layout(
    title='Trajetoria de crescimento do consumo de gasolina',
    xaxis_title='Tempo',
    yaxis_title='Valores (%)',
    legend_title='Séries',
    template='plotly_white'
)

# Exibição do gráfico
fig.show()

### Analise do consumo de diesel

In [41]:
# Consumo médio proporcional para Diesel
consumo_proporcional_diesel = (consumo_medio_diesel / consumo_medio_diesel.iloc[0])

# Ordenando os valores
consumo_proporcional_diesel = consumo_proporcional_diesel.sort_values('Valores (%)', ascending=False)

# Removendo o Brasil ('br') e ordenando novamente
consumo_proporcional_diesel = consumo_proporcional_diesel.drop('br').sort_values(ascending=False, by='Valores (%)')

# Ajustando os valores para percentual
consumo_proporcional_diesel = consumo_proporcional_diesel * 100

# Exibindo os resultados
consumo_proporcional_diesel

# Gráfico de barras para o consumo de Diesel
px.bar(consumo_proporcional_diesel, y='Valores (%)', title='Consumo de Diesel')

In [42]:
# Cálculo do crescimento do mercado para Diesel
crescimento_mercado_diesel = total_consumo_diesel / total_consumo_diesel.iloc[0]
crescimento_mercado_diesel

# Criando o gráfico
fig = go.Figure()

# Adicionando linhas para cada coluna de valores
for col in crescimento_mercado_diesel.columns[1:]:
    fig.add_trace(go.Scatter(
        x = crescimento_mercado_diesel.index,
        y = crescimento_mercado_diesel[col],
        mode='lines+markers',
        name=col
    ))

# Configuração do layout
fig.update_layout(
    title='Trajetória de Crescimento do Consumo de Diesel',
    xaxis_title='Tempo',
    yaxis_title='Valores (%)',
    legend_title='Séries',
    template='plotly_white'
)

# Exibição do gráfico
fig.show()

### Analise Etanol


In [43]:
# Consumo médio proporcional para Etanol
consumo_proporcional_etanol = (consumo_medio_etanol / consumo_medio_etanol.iloc[0])

# Ordenando os valores
consumo_proporcional_etanol = consumo_proporcional_etanol.sort_values('Valores (%)', ascending=False)

# Removendo o Brasil ('br') e ordenando novamente
consumo_proporcional_etanol = consumo_proporcional_etanol.drop('br').sort_values(ascending=False, by='Valores (%)')

# Ajustando os valores para percentual
consumo_proporcional_etanol = consumo_proporcional_etanol * 100

# Exibindo os resultados
consumo_proporcional_etanol

# Gráfico de barras para o consumo de Etanol
px.bar(consumo_proporcional_etanol, y='Valores (%)', title='Consumo de Etanol')

In [44]:
# Cálculo do crescimento do mercado para Etanol
crescimento_mercado_etanol = total_consumo_etanol / total_consumo_etanol.iloc[0]
crescimento_mercado_etanol

# Criando o gráfico
fig = go.Figure()

# Adicionando linhas para cada coluna de valores
for col in crescimento_mercado_etanol.columns[1:]:
    fig.add_trace(go.Scatter(
        x = crescimento_mercado_etanol.index,
        y = crescimento_mercado_etanol[col],
        mode='lines+markers',
        name=col
    ))

# Configuração do layout
fig.update_layout(
    title='Trajetória de Crescimento do Consumo de Etanol',
    xaxis_title='Tempo',
    yaxis_title='Valores (%)',
    legend_title='Séries',
    template='plotly_white'
)

# Exibição do gráfico
fig.show()